# Importing Libraries

In [1]:
import numpy as np
import os
import re
import gensim
import string
import nltk
from tqdm import tqdm
import tensorflow as tf
import keras
#import scipy.sparse import coo_array

2022-07-25 20:23:03.068550: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-25 20:23:03.072292: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-25 20:23:03.072308: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
print(os.listdir("/home/dasharath/Downloads/word2vec_dataset/word2vec_dataset/review_hotels/data"))

['dubai', 'beijing', 'san-francisco', 'new-york-city', 'london', 'las-vegas', 'new-delhi', 'shanghai', '.DS_Store', 'chicago', 'montreal']


In [3]:
reviews=[]
path="/home/dasharath/Downloads/word2vec_dataset/word2vec_dataset/review_hotels/data"
for folders in os.listdir(path):
    #print(folders)
    if not folders[0].isalnum():
        continue
    i=0
    for filename in os.listdir(path + '/' + folders):
        if(i>25):
            break
        with open(os.path.join(path + '/' + folders,filename),'r', errors='ignore') as f:
            for rv in f.readlines():
                reviews.append(rv)
        i+=1
len(reviews)
#reviews[5]

25307

# Data Pre-processing

In [4]:
valid_reviews=[]
null_reviews=[]
for i in range(len(reviews)):
    try:
        valid_reviews.append(reviews[i][reviews[i].index('\t')+1:]) #removing date and month from every review
    except:
         null_reviews.append(reviews[i])
len(valid_reviews)

25307

In [5]:
valid_reviews[5]

"Best beach hotel in DUbai\tWe enjoyed our HONEYMOON at the Habtoor Hotel. Talking about the location: it was the best beach hotel in the area called Dubai Marina. The zone is totaly new and the hotel's beach is the best in the area (we visited some good hotels).The service was 100% high.The room was very nice, with big nice bathroom, with a view to the infinity pool+sea+city. And it was just a standart room. It was clean all the time we stayed there, cleanig the room twice a day, serving fruits in the first day for free and chocolates every day for free. There are a lot of nice restaurants and bars on the hotel's territory.The food at te main restaurant was various. We liked we could choose from lots of fresh juices (orange, watermelon, carrot...) and the food was also fresh.What we didnt like is that the vacation was over and we had to leave :)If we go in Dubai again - we deffinetely choose Habtoor Grand.p.s. There was free bus trips to all the biggest malls in Dubai which really imp

In [6]:
null_reviews

[]

In [7]:
from nltk.stem import WordNetLemmatizer
stopwords=nltk.corpus.stopwords.words('english')
lemma=WordNetLemmatizer()

In [8]:
def pre_processing(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [lemma.lemmatize(word) for word in tokens if word not in stopwords and word.isalpha()]
    return text

In [9]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
word_lists=[]
from collections import Counter
all_words=Counter()
for i ,text in tqdm(enumerate(valid_reviews)):
    valid_reviews[i]=pre_processing(text)
    for word in  valid_reviews[i]:
        all_words[word]+=1

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/dasharath/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/dasharath/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
25307it [00:14, 1689.13it/s]


In [10]:
k=0
uniq_words=[]
for w,i in sorted(all_words.items(),key=lambda x:x[1],reverse=True):
    if k>10000:
        break
    uniq_words.append(w)
    k+=1

In [11]:
unique_word_dict = {}
for i, word in tqdm(enumerate(uniq_words)):
    unique_word_dict.update({
        word: i
    })

10001it [00:00, 1244318.90it/s]


In [12]:
for w,i in unique_word_dict.items():
    if i>20:
        break
    print(w,i) 

hotel 0
room 1
great 2
stay 3
good 4
would 5
staff 6
location 7
night 8
one 9
nice 10
time 11
stayed 12
u 13
clean 14
service 15
get 16
place 17
bed 18
day 19
breakfast 20


In [13]:
window= 4

word_lists = []

for j in tqdm(range(len(valid_reviews))):
    valid_reviews[j]=[word for word in valid_reviews[j] if word in uniq_words]
    for i, word in enumerate(valid_reviews[j]):
        for w in range(window):
            if i + 1 + w < len(valid_reviews[j]): 
                word_lists.append([word] + [valid_reviews[j][(i + 1 + w)]])    
            if i - w - 1 >= 0:
                word_lists.append([word] + [valid_reviews[j][(i - w - 1)]])

100%|████████████████████████████████████| 25307/25307 [00:41<00:00, 613.68it/s]


In [14]:
len(word_lists)

16496744

In [15]:
n=len(unique_word_dict)
n

10001

In [16]:
import keras
    from tensorflow.sparse import SparseTensor as spa

In [17]:
#x=spa(indices=[[i,unique_word_dict[word_lists[i][0]]] for i in range(128)],values=[1 for i in range(128)],dense_shape=[128,n])

In [19]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, word_lists, unique_word_dict, batch_size=128,shuffle=False):
        'Initialization'
        self.word_lists = word_lists
        self.batch_size = batch_size
        self.unique_word_dict = unique_word_dict
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.word_lists) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.word_lists[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.word_lists))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, word_lists_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        n=len(self.unique_word_dict)
        n1=len(word_lists_temp)
        
        X = spa(indices=[[i,self.unique_word_dict[word_lists_temp[i][0]]] for i in range(n1)],values=[1 for i in range(n1)],dense_shape=[n1,n])
        Y = spa(indices=[[i,self.unique_word_dict[word_lists_temp[i][1]]] for i in range(n1)],values=[1 for i in range(n1)],dense_shape=[n1,n])
        return X,tf.sparse.to_dense(Y)

In [20]:
from keras.models import Model
from keras.layers import Dense, Input
inp_out_nodes=len(unique_word_dict)
embed_size = 50

data_generator=DataGenerator(word_lists,unique_word_dict,batch_size=128)

inp = Input(shape=(inp_out_nodes,),sparse=True)
x = Dense(units=embed_size, activation='linear')(inp)
x = Dense(units=inp_out_nodes, activation='softmax')(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

model.fit_generator(
    generator=data_generator,
    use_multiprocessing=True,
    workers=6,
    epochs=4
    )

Epoch 1/4


2022-07-25 20:36:28.600894: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-25 20:36:28.600917: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-25 20:36:28.600934: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (linux): /proc/driver/nvidia/version does not exist
2022-07-25 20:36:28.601099: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/tmp/ipykernel_17247/3527372978.py:14: UserWarning: `Model.fit_generator` is deprecated and will 

128880/128880 [==============================] - 2511s 19ms/step - loss: 7.1658
Epoch 2/4
128880/128880 [==============================] - 2511s 19ms/step - loss: 7.1099
Epoch 3/4
128880/128880 [==============================] - 2549s 20ms/step - loss: 7.1132
Epoch 4/4
128880/128880 [==============================] - 2521s 20ms/step - loss: 7.1112


In [21]:
weights = model.get_weights()[0]
print(weights.shape)
# Creating a dictionary to store the embeddings in. The key is a unique word and 
# the value is the numeric vector
embedding_dict = {}
for word in uniq_words: 
    embedding_dict.update({
        word: weights[unique_word_dict.get(word)]
        })

(10001, 50)


In [22]:
from numpy.linalg import norm
def cossim(x,y):
    return np.dot(x,y)/norm(x)*norm(y)
def top(x):
    tops=[]
    for w,i in embedding_dict.items():
        tops.append([cossim(i,x),w])
    tops.sort(reverse=True)
    return tops[:10]

In [31]:
print(top(embedding_dict['polite']))

[[10.9499655, 'polite'], [10.1619625, 'courteous'], [9.837849, 'friendly'], [9.711471, 'professional'], [9.218165, 'knowledgeable'], [9.184785, 'welcoming'], [9.153411, 'accomodating'], [9.053712, 'staff'], [9.040276, 'efficient'], [8.98842, 'helpful']]


In [32]:
print(top(embedding_dict['dirty']))

[[9.337451, 'dirty'], [8.56621, 'filthy'], [8.040773, 'stained'], [7.9002037, 'stain'], [7.527453, 'carpet'], [7.330088, 'gross'], [7.1776714, 'vacuumed'], [7.084972, 'dust'], [7.0503836, 'moldy'], [7.0153537, 'nasty']]


In [33]:
print(top(embedding_dict['france']))

[[5.352672, 'france'], [3.0372612, 'draycott'], [2.9666753, 'singapore'], [2.9248126, 'england'], [2.8994017, 'leg'], [2.8889282, 'canada'], [2.812708, 'newbie'], [2.8088207, 'dream'], [2.803924, 'caribbean'], [2.7674694, 'paragon']]


In [36]:
print(top(embedding_dict['shock']))

[[5.495183, 'shock'], [3.137311, 'screwed'], [3.1298199, 'frazzled'], [3.1182957, 'glance'], [3.080535, 'mistake'], [3.007596, 'visa'], [2.9525337, 'cc'], [2.926972, 'impression'], [2.902279, 'realised'], [2.8975484, 'delayed']]
